In [1]:
# setup library imports
import io, time, json
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import re
from datetime import timedelta

In [2]:
def to_datetime(x):
    time_converted = datetime.strptime(x, '%B %d %Y %I:%M %p')
    return time_converted
def hour_rounder(t):
    # Rounds to nearest hour by adding a timedelta hour if minute >= 30
    return (t.replace(second=0, microsecond=0, minute=0, hour=t.hour)
               +timedelta(hours=t.minute//30))

In [3]:
def one_day(url):

    day = requests.get(url)

    soup = BeautifulSoup(day.text, 'html.parser')

    a=soup.find_all('div', attrs={'class':'high-res'})

    date = soup.find_all('h2', attrs={'class':"history-date"})
    date = date[0]
    date = date.text
    try:
        a=a[0]
        rows=[]
        b = a.find_all('tr')

        for i in range(1,len(b)):
            c = b[i].find_all('td')
            rows.append(c)
        e = b[0].find_all('th')
        cols=[]
        for header in e:
            cols.append(header.text)

        weather = pd.DataFrame(columns = ['Time', 'Temp (F)', 'Humidity', 'Wind Speed', 'Condition'])

        data = []
        for k in rows:
            row={}
            time = k[0].text
            time = date+' '+time
            time = re.sub(r'.*day,', '', time)
            time = time.replace(',','')
            time = time[1:]
            try:
                temp = k[1].find('span', attrs={'class':'wx-value'}).text
            except:
                temp = None
            if len(cols)==13:    
                hum = k[4].text
                hum = hum[:-1]
                try:
                    wind_spd = k[8].find('span', attrs={'class':'wx-value'}).text
                except:
                    wind_spd = k[8].text

                cond = k[12].text
            if len(cols)==12:
                hum = k[3].text
                hum = hum[:-1]
                try:
                    wind_spd = k[7].find('span', attrs={'class':'wx-value'}).text
                except:
                    wind_spd = k[7].text

                cond = k[11].text
            row["Time"]=time
            row['Temp (F)']=temp
            row['Humidity']=hum
            row['Wind Speed']=wind_spd
            row["Condition"]=cond
            data.append(row)

        weather = weather.append(data,ignore_index=True)
        try:
            weather['Temp (F)'] = weather['Temp (F)'].astype('float64')
        except:
            pass
        try:
            weather['Humidity'] = weather['Humidity'].astype('float64')
        except:
            pass
        weather['Time'] = weather['Time'].apply(lambda x: hour_rounder((pd.to_datetime(to_datetime(x)))))
        weather = weather.replace(['N/A','\n -\n'], 50.0)
        weather1 = weather.copy()
        #weather1 = weather1.replace(['N/A','\n  -\n'], 0.0)
        weather1['Humidity'] = weather1['Humidity'].astype('float64')
        weather1 = weather1.set_index('Time', drop=True)
        weather1 = weather1.resample('H').mean()
        weather1 = weather1.reset_index()

        n = soup.find_all('div', attrs={'class':'next-link'})
        n=n[0]
        nxt_date = 'https://www.wunderground.com'+n.find('a').get('href')

        return weather, weather1, nxt_date
    except:
        n = soup.find_all('div', attrs={'class':'next-link'})
        n=n[0]
        nxt_date = 'https://www.wunderground.com'+n.find('a').get('href')

        return None, None, nxt_date

In [4]:
result = one_day('https://www.wunderground.com/history/airport/KAGC/2002/1/1/DailyHistory.html?req_city=&req_state=&req_statename=&reqdb.zip=&reqdb.magic=&reqdb.wmo=')
result

(                  Time  Temp (F)  Humidity Wind Speed         Condition
 0  2002-01-01 01:00:00      18.0      74.0        4.6        Light Snow
 1  2002-01-01 02:00:00      18.0      74.0        8.1        Light Snow
 2  2002-01-01 03:00:00      18.0      74.0        9.2        Light Snow
 3  2002-01-01 04:00:00      17.1      77.0        8.1          Overcast
 4  2002-01-01 05:00:00      16.0      77.0        8.1     Partly Cloudy
 5  2002-01-01 06:00:00      14.0      84.0        8.1  Scattered Clouds
 6  2002-01-01 07:00:00      12.9      84.0        6.9  Scattered Clouds
 7  2002-01-01 08:00:00      12.0      88.0        8.1     Partly Cloudy
 8  2002-01-01 09:00:00      12.9      84.0        9.2     Partly Cloudy
 9  2002-01-01 10:00:00      16.0      74.0        8.1     Partly Cloudy
 10 2002-01-01 11:00:00      18.0      65.0        8.1     Partly Cloudy
 11 2002-01-01 12:00:00      19.9      65.0        8.1     Mostly Cloudy
 12 2002-01-01 13:00:00      21.0      63.0        

In [8]:
def all_days(url, no_of_days):
    list1 = []
    list2= []
    result = one_day(url)
    list1.append(result[0])
    list2.append(result[1])
    for i in range(no_of_days -1):
        if result!=None:
            result = one_day(result[2])
        
            list1.append(result[0])
            list2.append(result[1])
        #print(i+1, 'th day is done')
    weather1 = pd.concat(list1).reset_index(drop=True)
    weather2 = pd.concat(list2).reset_index(drop=True)
    return weather1, weather2
one, two = all_days('https://www.wunderground.com/history/airport/KAGC/2002/1/1/DailyHistory.html?req_city=&req_state=&req_statename=&reqdb.zip=&reqdb.magic=&reqdb.wmo=',31)

In [217]:
one_year = [1,2,3,4,5,6,7,8,9,10,11,12]
days_in_month = [31,28,31,30,31,30,31,31,30,31,30,31]
Month = list(zip(one_year,days_in_month))
all_months_1 = []
all_months_2 = []
for i in Month:
    url_initial = 'https://www.wunderground.com/history/airport/KAGC/2002/'+str(i[0])+'/1/DailyHistory.html?req_city=&req_state=&req_statename=&reqdb.zip=&reqdb.magic=&reqdb.wmo='
    res = all_days(url_initial,i[1])
    all_months_1.append(res[0])
    all_months_2.append(res[1])
    print('Month',i[0],'is done')
all_Months_1 = pd.concat(all_months_1).reset_index(drop=True)
all_Months_2 = pd.concat(all_months_1).reset_index(drop=True)
all_Months_1,all_Months_2

Month 1 is done
Month 2 is done
Month 3 is done
Month 4 is done
Month 5 is done
Month 6 is done
Month 7 is done
Month 8 is done
Month 9 is done
Month 10 is done
Month 11 is done
Month 12 is done


(                     Time  Temp (F) Humidity Wind Speed  \
 0     2002-01-01 01:00:00      36.0       64        6.9   
 1     2002-01-01 02:00:00      36.0       57        8.1   
 2     2002-01-01 03:00:00      36.0       57        8.1   
 3     2002-01-01 04:00:00      36.0       57        5.8   
 4     2002-01-01 05:00:00      35.1       61        8.1   
 5     2002-01-01 06:00:00      34.0       64        4.6   
 6     2002-01-01 07:00:00      35.1       64        6.9   
 7     2002-01-01 08:00:00      34.0       64        9.2   
 8     2002-01-01 09:00:00      35.1       64        8.1   
 9     2002-01-01 10:00:00      37.0       59       10.4   
 10    2002-01-01 11:00:00      39.0       55        6.9   
 11    2002-01-01 12:00:00      39.0       55        6.9   
 12    2002-01-01 13:00:00      39.0       57        5.8   
 13    2002-01-01 14:00:00      39.9       58       11.5   
 14    2002-01-01 15:00:00      41.0       57        8.1   
 15    2002-01-01 16:00:00      39.9    

In [58]:
Jan_to_Jul_all = pd.concat(all_months_1).reset_index(drop=True)
Jan_to_Jul = pd.concat(all_months_2).reset_index(drop=True)
Jan_to_Jul_all.to_csv('Jan_to_Jul_all_2015.csv')
Jan_to_Jul.to_csv('Jan_to_Jul_2015.csv')

In [70]:
l_1,l_2 = all_days('https://www.wunderground.com/history/airport/KAGC/2015/8/12/DailyHistory.html?req_city=&req_state=&req_statename=&reqdb.zip=&reqdb.magic=&reqdb.wmo=',2)

1 th day is done


In [131]:
all_Months_1

,Time,Temp (F),Humidity,Wind Speed,Condition
0,2014-01-01 01:00:00,39.9,68,Calm,Clear
1,2014-01-01 02:00:00,39.0,73,Calm,Clear
2,2014-01-01 03:00:00,39.0,70,3.5,Clear
3,2014-01-01 04:00:00,39.9,68,Calm,Scattered Clouds
4,2014-01-01 05:00:00,39.9,68,4.6,Clear
5,2014-01-01 06:00:00,39.9,68,3.5,Mostly Cloudy
6,2014-01-01 07:00:00,39.0,70,Calm,Clear
7,2014-01-01 08:00:00,39.0,70,Calm,Clear
8,2014-01-01 09:00:00,46.0,58,4.6,Clear
9,2014-01-01 10:00:00,52.0,59,6.9,Clear


In [187]:
list1, list2 = all_day('https://www.wunderground.com/history/airport/KATT/2016/2/1/DailyHistory.html?req_city=&req_state=&req_statename=&reqdb.zip=&reqdb.magic=&reqdb.wmo=',29)

1 th day is done
2 th day is done
3 th day is done
4 th day is done
5 th day is done
6 th day is done
7 th day is done
8 th day is done
9 th day is done
10 th day is done
11 th day is done
12 th day is done
13 th day is done
14 th day is done
15 th day is done
16 th day is done
17 th day is done
18 th day is done
19 th day is done
20 th day is done
21 th day is done
22 th day is done
23 th day is done
24 th day is done
25 th day is done
26 th day is done
27 th day is done
28 th day is done


In [188]:
list2

,Time,Temp (F),Humidity
0,2016-02-01 01:00:00,60.10,49.000000
1,2016-02-01 02:00:00,57.00,55.000000
2,2016-02-01 03:00:00,55.90,57.000000
3,2016-02-01 04:00:00,55.95,72.250000
4,2016-02-01 05:00:00,55.90,97.000000
5,2016-02-01 06:00:00,55.90,97.000000
6,2016-02-01 07:00:00,56.45,100.000000
7,2016-02-01 08:00:00,57.00,100.000000
8,2016-02-01 09:00:00,58.45,98.000000
9,2016-02-01 10:00:00,62.55,85.000000


In [189]:
#list2.to_csv('Dec 2017.csv')
list1.to_csv('Feb 2016_all.csv')

In [227]:
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
years = [2016, 2017]
all_years_1 = []
all_years_2 = []
for month in months:
    for year in years:
        one = pd.read_csv(str(month)+' '+str(year)+'.csv', sep=',', delimiter=None, header='infer')
        two = pd.read_csv(str(month)+' '+str(year)+'_all.csv', sep=',', delimiter=None, header='infer')
        all_years_1.append(one)
        all_years_2.append(two)
Weather_data_one = pd.concat(all_years_1).reset_index()
#Weather_data_one = Weather_data_one.drop(['index', 'Unnamed: 0'], axis=1)
Weather_data_two = pd.concat(all_years_2).reset_index()
#Weather_data_two = Weather_data_two.drop(['index', 'Unnamed: 0'], axis=1)

In [30]:
#Weather_data_one['Time'] = Weather_data_one['Time'].apply(lambda x: pd.to_datetime(x))
# Weather_data_one = Weather_data_one.drop(['index', 'Unnamed: 0'], axis=1)
# Weather_data_two = Weather_data_two.drop(['index', 'Unnamed: 0'], axis=1)
Weather_data_one.to_csv('Weather data.csv')
Weather_data_two.to_csv('Weather data All.csv')

NameError: name 'Weather_data_one' is not defined

In [248]:
X = df_2015_1
X

,Time,Temp (F),Humidity,Wind Speed,Condition
0,2015-01-01 01:00:00,37.9,76.0,Calm,Overcast
1,2015-01-01 02:00:00,37.9,70.0,4.6,Overcast
2,2015-01-01 03:00:00,37.0,73.0,4.6,Overcast
3,2015-01-01 04:00:00,37.0,73.0,4.6,Overcast
4,2015-01-01 05:00:00,35.1,92.0,6.9,Light Thunderstorms and Rain
5,2015-01-01 05:00:00,35.1,92.0,4.6,Rain
6,2015-01-01 05:00:00,34.0,96.0,4.6,Heavy Rain
7,2015-01-01 05:00:00,34.0,96.0,4.6,Heavy Thunderstorms and Rain
8,2015-01-01 06:00:00,34.0,96.0,5.8,Rain
9,2015-01-01 06:00:00,34.0,100.0,3.5,Rain


In [249]:
y = X.Condition.unique().tolist()
y = sorted(y)
X['Condition'] = X['Condition'].apply(lambda x: y.index(x))
# y2=[]
# for i in range(len(X)):
    
#     try:
#         X['Wind Speed'][i]=float(X['Wind Speed'][i])
#     except:
#         X['Wind Speed'][i]=X['Wind Speed'][i]
# for x in X['Wind Speed']:
#     if type(x)==str:
#         y2.append(x)
# y2
def t_or_e(x):
    try:
        return float(x)
    except:
        if x=='Calm':
            return 3.0
        else:
            return 4.5
X['Wind Speed'] = X['Wind Speed'].apply(lambda x: t_or_e(x))


In [250]:
X['Time'] = X['Time'].apply(lambda x: pd.to_datetime(x))
X.dtypes

Time          datetime64[ns]
Temp (F)             float64
Humidity             float64
Wind Speed           float64
Condition              int64
dtype: object

In [251]:
X['Humidity'] = X['Humidity'].astype('float64')
#X['Humidity'] = X['Humidity'].apply(lambda x: x/100)
X['Humidity'] = X['Humidity'].apply(lambda x: x/100)

#X.to_csv('Final Data.csv')
X = X.set_index('Time', drop=True).resample('H').mean().reset_index()

In [253]:
X.to_csv('2015_1.csv')

In [252]:
X

,Time,Temp (F),Humidity,Wind Speed,Condition
0,2015-01-01 01:00:00,37.900000,0.7600,3.000000,10.000000
1,2015-01-01 02:00:00,37.900000,0.7000,4.600000,10.000000
2,2015-01-01 03:00:00,37.000000,0.7300,4.600000,10.000000
3,2015-01-01 04:00:00,37.000000,0.7300,4.600000,10.000000
4,2015-01-01 05:00:00,34.550000,0.9400,5.175000,6.500000
5,2015-01-01 06:00:00,34.000000,0.9760,4.080000,12.000000
6,2015-01-01 07:00:00,34.366667,0.9600,3.700000,6.000000
7,2015-01-01 08:00:00,35.100000,0.9300,4.350000,8.250000
8,2015-01-01 09:00:00,35.350000,0.9250,5.800000,10.000000
9,2015-01-01 10:00:00,35.100000,0.9600,5.800000,10.000000


In [254]:
df_2002 = pd.read_csv('2002_all.csv', sep=',', delimiter=None, header='infer').drop('Unnamed: 0', axis=1)
df_2003 = pd.read_csv('2003_all.csv', sep=',', delimiter=None, header='infer').drop('Unnamed: 0', axis=1)
df_2004 = pd.read_csv('2004_all.csv', sep=',', delimiter=None, header='infer').drop('Unnamed: 0', axis=1)
df_2005 = pd.read_csv('2005_all.csv', sep=',', delimiter=None, header='infer').drop('Unnamed: 0', axis=1)
df_2006 = pd.read_csv('2006_all.csv', sep=',', delimiter=None, header='infer').drop('Unnamed: 0', axis=1)
df_2007 = pd.read_csv('2007_all.csv', sep=',', delimiter=None, header='infer').drop('Unnamed: 0', axis=1)
df_2008 = pd.read_csv('2008_all.csv', sep=',', delimiter=None, header='infer').drop('Unnamed: 0', axis=1)
df_2009 = pd.read_csv('2009_all.csv', sep=',', delimiter=None, header='infer').drop('Unnamed: 0', axis=1)
df_2010 = pd.read_csv('2010_all.csv', sep=',', delimiter=None, header='infer').drop('Unnamed: 0', axis=1)
df_2011 = pd.read_csv('2011_all.csv', sep=',', delimiter=None, header='infer').drop('Unnamed: 0', axis=1)
df_2012 = pd.read_csv('2012_all.csv', sep=',', delimiter=None, header='infer').drop('Unnamed: 0', axis=1)
df_2013 = pd.read_csv('2013_all.csv', sep=',', delimiter=None, header='infer').drop('Unnamed: 0', axis=1)
df_2014 = pd.read_csv('2014_all.csv', sep=',', delimiter=None, header='infer').drop('Unnamed: 0', axis=1)
df_2015_1 = pd.read_csv('2015_1.csv', sep=',', delimiter=None, header='infer').drop('Unnamed: 0', axis=1)
df_2015_2 = pd.read_csv('Aug_to_Dec_2015_all.csv', sep=',', delimiter=None, header='infer').drop('Unnamed: 0', axis=1)
df_2016_17 = pd.read_csv('Final Data.csv', sep=',', delimiter=None, header='infer').drop('Unnamed: 0', axis=1)

In [255]:
df_weather=[]
df_weather.append(df_2002)
df_weather.append(df_2003)
df_weather.append(df_2004)
df_weather.append(df_2005)
df_weather.append(df_2006)
df_weather.append(df_2007)
df_weather.append(df_2008)
df_weather.append(df_2009)
df_weather.append(df_2010)
df_weather.append(df_2011)
df_weather.append(df_2012)
df_weather.append(df_2013)
df_weather.append(df_2014)
df_weather.append(df_2015_1)
df_weather.append(df_2015_2)
df_weather.append(df_2016_17)


In [259]:
Weather_Data = pd.concat(df_weather).reset_index(drop=True)
try:
    Weather_Data['Condition'] = Weather_Data['Condition'].astype('int64')
except:
    pass
Weather_Data.to_csv('Weather Data.csv')

In [256]:
df_2015_1

,Time,Temp (F),Humidity,Wind Speed,Condition
0,2015-01-01 01:00:00,37.900000,0.7600,3.000000,10.000000
1,2015-01-01 02:00:00,37.900000,0.7000,4.600000,10.000000
2,2015-01-01 03:00:00,37.000000,0.7300,4.600000,10.000000
3,2015-01-01 04:00:00,37.000000,0.7300,4.600000,10.000000
4,2015-01-01 05:00:00,34.550000,0.9400,5.175000,6.500000
5,2015-01-01 06:00:00,34.000000,0.9760,4.080000,12.000000
6,2015-01-01 07:00:00,34.366667,0.9600,3.700000,6.000000
7,2015-01-01 08:00:00,35.100000,0.9300,4.350000,8.250000
8,2015-01-01 09:00:00,35.350000,0.9250,5.800000,10.000000
9,2015-01-01 10:00:00,35.100000,0.9600,5.800000,10.000000
